In [1020]:
import json
import numpy as np

In [1021]:
def linear(x):
    return x

def derivative_linear(x):
    return np.ones(x.shape)

def relu(x):
    return np.maximum(0, x)

def derivative_relu(x):
    return np.where(x < 0, 0, 1)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def derivative_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def derivative_softmax(x, isTarget):
    if isTarget:
        return softmax(x)
    else:
        return -(1-softmax(x))

In [1022]:
def delta_output(activation_function, target, output, net):
  if activation_function == "linear":
      result = (target - output) * derivative_linear(net)
  elif activation_function == "relu":
      result = (target - output) * derivative_relu(net)
  elif activation_function == "sigmoid":
      result = (target - output) * derivative_sigmoid(net)
  elif activation_function == "softmax":
      result = -derivative_softmax(net, True)

  return result

def delta_hidden(activation_function, net, delta_output, weights):
  if activation_function == "linear":
      result = np.dot(delta_output, weights.T) * derivative_linear(net)
  elif activation_function == "relu":
      result = np.dot(delta_output, weights.T) * derivative_relu(net)
  elif activation_function == "sigmoid":
      result = np.dot(delta_output, weights.T) * derivative_sigmoid(net)
  elif activation_function == "softmax":
      result = -derivative_softmax(net, False)

  return result

def calculate_delta_weight(activation_function, learning_rate, target, output, net, weights, x, isOutputLayer):
    if isOutputLayer:
        return learning_rate * delta_output(activation_function, target, output, net) * x
    else:
        return learning_rate * delta_hidden(activation_function, net, delta_output, weights) * x

In [1023]:
def loss_function_sse(target, output):
    # print("loss function sse target: ", target)
    # print("loss function sse output: ", output)
    
    e = 0.5 * np.sum((target - output) ** 2)
    return e

def loss_function_softmax(target, output):
    e = -np.sum(target * np.log(output))
    return e

In [1024]:
def terminate_condition(stopped_by, max_iteration, error_threshold, iteration, error):
    if stopped_by == "max_iteration":
        return iteration >= max_iteration
    elif stopped_by == "error_threshold":
        return error <= error_threshold

def forward_propagation(model, input_data, weights):
    layers = model["layers"]
    output_layer = [input_data]

    for i, layer in enumerate(layers):
        activation_function = layer["activation_function"]
        weight_matrix = weights[i]
        bias = layer.get("bias", 1)

        input_activation = output_layer[-1]
        # print("Dimensi input_activation:", input_activation.shape)
        # print(input_activation)
        if len(input_activation.shape) == 1:
            input_activation_with_bias = np.insert(input_activation, 0, bias)
            input_activation_with_bias = input_activation_with_bias.reshape(1, -1)
        else:
            batch_size = input_activation.shape[0]
            bias_vector = np.ones((batch_size, 1)) * bias
            input_activation_with_bias = np.concatenate((bias_vector, input_activation), axis=1)
        # print("Dimensi input_activation_with_bias:", input_activation_with_bias.shape)
        # print(input_activation_with_bias)
        # print("Dimensi weight_matrix:", weight_matrix.shape)
        # print(weight_matrix)

        output_linear_combination = np.dot(input_activation_with_bias, weight_matrix)

        if activation_function == "linear":
            activation_result = linear(output_linear_combination)
        elif activation_function == "relu":
            activation_result = relu(output_linear_combination)
        elif activation_function == "sigmoid":
            activation_result = sigmoid(output_linear_combination)
        elif activation_function == "softmax":
            activation_result = softmax(output_linear_combination)

        output_layer.append(activation_result)

    return output_layer

def back_propagation(model, input, initial_weights, target_input, learning_rate, batch_size, stopped_by, max_iteration, error_threshold):

    epoch=0
    error = np.inf

    while not terminate_condition(stopped_by, max_iteration, error_threshold, epoch, error) :
        epoch += 1

        if error == np.inf:
            error = 0

        for batch in range(int(input.shape[0]/batch_size)):
            input_batch = input[batch*batch_size:(batch+1)*batch_size]
            target_batch = target_input[batch*batch_size:(batch+1)*batch_size]
            output_forward = forward_propagation(model, input_batch, initial_weights)
            
            delta_weight = [np.zeros(w.shape) for w in initial_weights]
            for id_data in range(batch_size):
                # Calculate error each output unit
                error_k = []
                id = id_data + batch_size * batch

                '''
                if (model["layers"][-1]["activation_function"] == "softmax"):
                    input_x = np.insert(output_forward[-2], 0, 1)
                    all_net = np.dot(input_x, initial_weights[-1].T)
                '''
                
                # Iterate through each output unit
                for i in range(len(output_forward[-1][id_data])):
                    activation_function = model["layers"][-1]["activation_function"]
                    output = output_forward[-1][id_data][i]
                    target = target_batch[id_data][i]

                    # print("output:", output) 
                    # print("output_forward :", output_forward)
                    # print("initial_weights:", initial_weights[-1][:,i])
                    
                    input_x = output_forward[-2][id_data]
                    input_x = np.insert(input_x, 0, 1)
                    # print("input x:", input_x)
                    
                    net = np.dot(input_x.T, initial_weights[-1][:,i])

                    weights = initial_weights[-1]

                    error_k_j = []
                    for j in range(input_x.shape[0]):
                        x = input_x[j]
                        
                        if (activation_function == "softmax"):
                            error_k_j.append(calculate_delta_weight(activation_function, learning_rate, target, output, all_net, weights, x, True))
                        else:
                            error_k_j.append(calculate_delta_weight(activation_function, learning_rate, target, output, net, weights, x, True))

                    error_k.append(error_k_j)
                        
                # print("error_k Inside:", error_k)

                # Calculate error each hidden unit
                error_h = []
                
                # Iterate through each hidden layer
                for j in range(len(output_forward)-2, 1, -1):
                    error_h_j = []
                    activation_function = model["layers"][j]["activation_function"]

                    # Iterate through each hidden layer unit
                    for k in range(len(output_forward[j][id_data])):
                        output = output_forward[j][id_data][k]
                        target = target_batch[id_data][k]

                        # print("output:", output) 
                        # print("output_forward :", output_forward)
                        # print("initial_weights:", initial_weights[j][:,k])
                        
                        input_x = output_forward[j-1][id_data]
                        input_x = np.insert(input_x, 0, 1)
                        # print("input x:", input_x)
                        
                        net = np.dot(input_x.T, initial_weights[j][:,k])

                        weights = initial_weights[j]

                        error_h_j_k = []
                        for l in range(input_x.shape[0]):
                            x = input_x[l]
                            
                            if (activation_function == "softmax"):
                                error_h_j_k.append(calculate_delta_weight(activation_function, learning_rate, target, output, all_net, weights, x, False))
                            else:
                                error_h_j_k.append(calculate_delta_weight(activation_function, learning_rate, target, output, net, weights, x, False))

                        error_h_j.append(error_h_j_k)
                    error_h.append(error_h_j)

                # Update delta weight
                # print ("error_k:", error_k)
                # print ("delta_weight:", delta_weight)

                for j in range(len(initial_weights)-1, -1, -1):
                    if j == len(initial_weights)-1:
                        for k in range(len(initial_weights[j])):
                            for l in range(len(initial_weights[j][k])):
                                # print("j:", j)
                                # print("k:", k)
                                # print("l:", l)
                                # print("error_k[l][k]:", error_k[l][k])
                                # print("delta_weight[j][k][l]:", delta_weight[j][k][l])
                                
                                delta_weight[j][k][l] += error_k[l][k]
                    else:
                        for k in range(len(initial_weights[j])):
                            for l in range(len(initial_weights[j][k])):
                                delta_weight[j][k][l] += error_h[j][k][l]

            # Update weight per batch
            # sum delta weight
            for i in range(len(initial_weights)):
                for j in range(len(initial_weights[i])):
                    for k in range(len(initial_weights[i][j])):
                        initial_weights[i][j][k] += delta_weight[i][j][k]

            # print(" Weights: ", initial_weights)
            # print("Delta Weights: ", delta_weight)

        # Calculate error
        for data in range(input.shape[0]):
            # print("Data: ", data)
            # print("target_input", target_input)
            # print("input_data", input)
            
            output_forward = forward_propagation(model, input, initial_weights)
            # print("output_forward", output_forward)

            activation_function = model["layers"][-1]["activation_function"]
            if activation_function == "softmax":
                error += loss_function_softmax(target_input[data], output_forward[-1][data])
            else:
                error += loss_function_sse(target_input[data], output_forward[-1][data])

    # print("Final: ", initial_weights)
    return initial_weights
                

In [1025]:
# Load JSON input
with open("test/backpropagation/iris_1.json", "r") as file:
    data = json.load(file)

# Extract data
model = data["case"]["model"]
input_data = np.array(data["case"]["input"])
initial_weights = [np.array(layer_weights) for layer_weights in data["case"]["initial_weights"]]
target = np.array(data["case"]["target"])
learning_rate = data["case"]["learning_parameters"]["learning_rate"]
batch_size = data["case"]["learning_parameters"]["batch_size"]
max_iteration = data["case"]["learning_parameters"]["max_iteration"]
error_threshold = data["case"]["learning_parameters"]["error_threshold"]
stopped_by = data["expect"]["stopped_by"]
final_weights = [np.array(layer_weights) for layer_weights in data["expect"]["final_weights"]]

new_weight = back_propagation(model, input_data, initial_weights, target, learning_rate, batch_size, stopped_by, max_iteration, error_threshold)
print("new weight: ", new_weight)
output = forward_propagation(model, input_data, new_weight)
print("output: ", output[-1])

new weight:  [array([[-0.36946807, -0.36706698, -0.68519825],
       [ 1.5506238 , -1.14787811, -0.93440806],
       [-1.15069189,  1.55188113, -0.93141998],
       [-0.74939998, -0.75107   ,  1.20062979],
       [ 0.01      ,  0.01      ,  0.01      ]])]
output:  [[0.76515554 0.18020709 0.16525917]
 [0.17943797 0.7658123  0.16567179]
 [0.24622131 0.24635702 0.62607889]
 [0.76515554 0.18020709 0.16525917]
 [0.17943797 0.7658123  0.16567179]]


In [1026]:
# output_layer = forward_propagation(model, input_data, initial_weights)
# print(output_layer)

In [1027]:
# backward propagation with sklearn
from sklearn.neural_network import MLPClassifier

In [1028]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

# Load dataset
dataset = pd.read_csv("test/backpropagation/iris.csv")
dataset = dataset.drop(columns=["Id"])
dataset = dataset.sample(frac=1).reset_index(drop=True)  # Shuffling the dataset
learning_data = dataset.drop(columns=["Species"])
target_data = dataset["Species"]
mlp = MLPClassifier(hidden_layer_sizes=(5,9,3), max_iter=1000, activation='logistic', learning_rate='constant', learning_rate_init=0.01,)
mlp.fit(learning_data, target_data)

y_pred = mlp.predict(learning_data)
print("Predicted values:")
print(y_pred)
print("Actual values:")
print(target_data)
print("Accuracy:", mlp.score(learning_data, target_data))

Predicted values:
['Iris-virginica' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-virginica' 'Iris-versicolor' 'Iris-virginica'
 'Iris-virginica' 'Iris-versicolor' 'Iris-virginica' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-virginica'
 'Iris-versicolor' 'Iris-virginica' 'Iris-setosa' 'Iris-setosa'
 'Iris-virginica' 'Iris-versicolor' 'Iris-setosa' 'Iris-versicolor'
 'Iris-virginica' 'Iris-virginica' 'Iris-versicolor' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-virginica' 'Iris-setosa'
 'Iris-virginica' 'Iris-setosa' 'Iris-virginica' 'Iris-virginica'
 'Iris-virginica' 'Iris-versicolor' 'Iris-versicolor' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-virginica' 'Iris-virginica' 'Iris-versicolor'
 'Iris-virginica' 'Iris-virginica' 'Iris-setosa' 'Iris-setosa'
 'Iris-virginica' 'Iris-virginica' 'Iris-setosa' 'Iris-versicolor'
 'Iris-setosa' 'Iris-versicol

In [1029]:
# Load JSON input
with open("test/backpropagation/iris.json", "r") as file:
    data = json.load(file)

# Extract data
model = data["case"]["model"]
initial_weights = [np.array(layer_weights) for layer_weights in data["case"]["initial_weights"]]
learning_rate = data["case"]["learning_parameters"]["learning_rate"]
batch_size = data["case"]["learning_parameters"]["batch_size"]
max_iteration = data["case"]["learning_parameters"]["max_iteration"]
error_threshold = data["case"]["learning_parameters"]["error_threshold"]
stopped_by = data["expect"]["stopped_by"]
final_weights = [np.array(layer_weights) for layer_weights in data["expect"]["final_weights"]]

# iris with implemented code
input_data = learning_data.to_numpy()
target = []
for i in range(len(target_data)):
    if target_data[i] == "Iris-setosa":
        target.append([1,-1,-1])
    elif target_data[i] == "Iris-versicolor":
        target.append([-1,1,-1])
    else:
        target.append([-1,-1,1])
target = np.array(target)

new_weight = back_propagation(model, input_data, initial_weights, target, learning_rate, batch_size, stopped_by, max_iteration, error_threshold)
print("new weight: ", new_weight)
output = forward_propagation(model, input_data, new_weight)
print("output: ", output[-1])
# print(output[-1])
# print(target)

count = 0
for i in range(len(output[-1])):
    max = -99000
    for j in range(3):
        if output[-1][i][j] > max:
            max = output[-1][i][j]
            idx = j
    
    if (idx == 0) : 
        temp = [1,-1,-1]
    elif (idx == 1):
        temp = [-1,1,-1]
    else:
        temp = [-1,-1,1]

    if (np.array_equal(temp, target[i])) :
        print(temp,target[i])
        count += 1
print("Accuracy: ", count/len(output[-1]))
    

new weight:  [array([[ 0.30183608, -0.09826237, -0.11831176],
       [ 0.22773577, -0.91320266, -0.99410488],
       [ 1.30754304, -0.5886788 , -0.58624191],
       [-2.22100868, -0.25857407, -0.28975625],
       [-0.93139097,  0.0025303 ,  0.0309893 ]])]
output:  [[4.90912823e-04 2.07205895e-04 1.12328769e-04]
 [2.63197005e-03 3.50646590e-04 2.02894190e-04]
 [7.50645754e-03 3.21595635e-04 1.79710329e-04]
 [9.69158032e-04 9.40748784e-05 4.85125437e-05]
 [4.73105715e-03 1.20273758e-04 6.31665691e-05]
 [1.22457343e-04 1.36828141e-05 6.19401417e-06]
 [6.83128010e-03 3.50197646e-04 1.97379105e-04]
 [2.53813089e-04 2.34975693e-04 1.32475548e-04]
 [3.55670686e-04 8.94696099e-05 4.66403757e-05]
 [7.84152146e-03 4.57374837e-04 2.62010214e-04]
 [6.75168328e-05 8.79021980e-05 4.64617325e-05]
 [9.74643827e-01 2.53332780e-04 1.52771773e-04]
 [9.22924069e-01 4.24587466e-04 2.60405138e-04]
 [9.06631845e-01 6.00040228e-04 3.69888396e-04]
 [8.22994726e-01 7.94862096e-04 5.00233136e-04]
 [2.05313073e-0